In [1]:
import plaidml.keras
plaidml.keras.install_backend()

In [2]:
import tensorflow as tf
import keras

# import other stuff
from keras import backend as K
import numpy as np

In [3]:
import pandas as pd

data=pd.read_csv("fer2013.csv")

In [4]:
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dropout,MaxPooling2D,Dense
from sklearn.model_selection import train_test_split

In [5]:
data.head(10)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
5,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
6,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
7,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
8,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training
9,2,255 254 255 254 254 179 122 107 95 124 149 150...,Training


In [6]:
num_features = 64
num_labels = 7
width, height = 48, 48
pixels = data['pixels'].tolist() 

faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')] 
    face=np.asarray(face).reshape(width,height)
    faces.append(face)
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1) 

emotions = pd.get_dummies(data['emotion']).to_numpy()

In [7]:
print(faces.shape)
print(emotions.shape)

(35887, 48, 48, 1)
(35887, 7)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(faces,emotions, test_size=0.2, random_state=42)


In [9]:
X_train = np.array(X_train, 'float32')
y_train = np.array(y_train, 'float32')
X_test = np.array(X_test, 'float32')
y_test = np.array(y_test, 'float32')

X_train=X_train/255
X_test=X_test/255

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_train = X_train.astype('float32')
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)
X_test = X_test.astype('float32')

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

28709 train samples
7178 test samples


In [10]:
print(len(y_train.shape))

2


In [11]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
import cv2

In [12]:
batch_size = 256
num_epochs = 30
input_shape = (48, 48, 1)
verbose = 1
num_classes = 7
 
"""data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)
 """
gen = ImageDataGenerator()
train_generator = gen.flow(X_train, y_train, batch_size=batch_size)

model = Sequential()

model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
 


INFO:plaidml:Opening device "opencl_amd_iceland.0"


In [13]:
# num_epochs = 1
model.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=num_epochs)

Epoch 1/1
113/113 [==============================] - 83s 733ms/step - loss: 1.8228 - acc: 0.2446


In [14]:
predictions=model.predict(X_test)
acc = model.evaluate(X_test, y_test)
print(acc[1])

7178/7178 [==============================] - 15s 2ms/step
0.2458902201191002


In [15]:
print(X_test.shape)

(7178, 48, 48, 1)


In [17]:
model.save('my_models.h5') 

In [17]:
from keras.models import load_model
import cv2
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}

# model = load_model('my_models.h5')


cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
#     cv2.imshow("img",frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img1 = cv2.resize(roi_gray, (48, 48))
        yp=cropped_img1.reshape((1,48,48,1))
        prediction = model.predict(yp)
        cv2.putText(frame, emotion_dict[int(np.argmax(prediction))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()